# Scope
This document covers the data layer representation of data, and is intended to be used over any lower level transport layer.

# Packet Structure
| Offset     | Type   | Description        |
|------------|--------|--------------------|
| 0x0000     | U8     | Sync Char (0xE4)   |
| 0x0001     | U8     | Sync Char (0xEB)   |
| 0x0002     | B[16]  | Source UUID        |
| 0x0012     | B[16]  | Destination UUID   |
| 0x0022     | U8     | Packet Class       |
| 0x0023     | U8     | Packet ID          |
| 0x0024     | U16    | Payload Length (N) |
| 0x0026     | U16    | Header Checksum    |
| 0x0028     | B[N]   | Payload            |
| 0x0028 + N | U16    | Packet Checksum    |
| 0x002A + N |        |                    |

# Header Checksum
The checksum shall be calculated over the entirety of the packet header (bytes 0x0000 to 0x0025) using the CRC-16 checksum (polynomial 0x1021, starting value 0xFFFF, no reflection).  The checksum itself shall be stored as a 16 bit big endian integer.

# Packet Checksum
The checksum shall be calculated over the entirety of the packet header and payload using the CRC-16 checksum (polynomial 0x1021, starting value 0xFFFF, no reflection).  The checksum itself shall be stored as a 16 bit big endian integer.  Due to the nature of CRCs, this is equivalent to computing the CRC-16 checksum over the entirety of the payload.

# Data Types
| Type | Length (bytes) | Description                    |
|------|----------------|--------------------------------|
| U8   | 1              | Unsigned 8 bit integer         |
| U16  | 2              | Unsigned 16 bit integer        |
| U32  | 4              | Unsigned 32 bit integer        |
| U64  | 8              | Unsigned 64 bit integer        |
| S8   | 1              | Signed 8 bit integer           |
| S16  | 2              | Signed 16 bit integer          |
| S32  | 4              | Signed 32 bit integer          |
| S64  | 8              | Signed 64 bit integer          |
| F32  | 4              | IEEE single precision float    |
| F64  | 8              | IEEE double precision float    |
| C[N] | N              | N-array of signed 8-bit chars  |
| B[N] | N              | N-array of unsigned 8-bit ints |

# Packet Classes
| Packet Class | Description           |
|--------------|-----------------------|
| 0x01         | Status Packets        |
| 0x02         | Configuration Packets |
| 0x03         | Command Packets       |
| 0x04         | Upgrade Packets       |
| 0x05         | Data Packets          |
| 0x06         | Interface Packets     |
| 0xFF         | Debug/Dev Packets     |

## Status (Class 0x01)

### Heartbeat Packet (ID 0x01)

## Configuration (Class 0x02)
Messages in the Configuration class can be used to configure the endpoint and poll current configuration values.  To set a device's configuration, first send the Set Configuration command (0x03 0x00).  Wait for the device to acknowledge the command, then within 30 seconds, send the new Configuration.  Expect an ACK within 30 seconds of sending the new Configuration.  To poll a device's configuration, send the Configuration packet with a payload of 0 bytes.  

### IMU Data Stream (0x00)
| Payload Offset | Type   | Description                       |
|----------------|--------|-----------------------------------|
| 0x0000         | U8     | Packet Version = 1                |
| 0x0001         | U8     | Coordinate Frame                  |
| 0x0002         | U16    | Nominal Sample Rate (samples/sec) |
| 0x0004         |        |                                   |

### Audio Data Stream (0x01)
| Payload Offset | Type   | Description                       |
|----------------|--------|-----------------------------------|
| 0x0000         | U8     | Packet Version = 1                |
| 0x0001         | U8     | Bit Depth                         |
| 0x0002         | U32    | Nominal Sample Rate (samples/sec) |
| 0x0004         | U16    | Channel Bitmask                   |
| 0x0006         |        |                                   |

### Raw Data Streams (0xF0)
This stores the raw data strem configuration.  Each MIME type should describe the custom data stream, and should be registered with both source and destination.  If the sensor does not support a particular MIME type, it should NACK the entire configuration.

| Payload Offset | Type   | Description            |
|----------------|--------|------------------------|
| 0x0000         | U8     | Packet Version         |
| 0x0001         | U8     | Number of Data IDs (N) |
| 0x0002         | C[256] | Data ID 0 MIME Type    |
| 0x0102         | C[256] | Data ID 1 MIME Type    |
| ...            |        |                        |
| 0x0002 + 256*N |        |                        |

## Command (Class 0x03)

### Set Configuration Command (0x00)
To set a device's configuration, first send this command.  Wait for the device to acknowledge the command, then within 30 seconds, send the new Configuration.  Expect an ACK within 30 seconds of sending the new Configuration.

| Payload Offset | Type | Description        |
|----------------|------|--------------------|
| 0x0000         | U8   | Packet Version = 1 |
| 0x0001         | U8   | Reserved           |
| 0x0002         |      |                    |

## ACK/NACK Packet (ID 0x00)

Acknowledge or reject command or configuration messages.

| Payload Offset | Type | Description        |
|----------------|------|--------------------|
| 0x0000         | U8   | Packet Version = 1 |
| 0x0001         | U8   | ACK (1) / NACK (0) |
| 0x0002         | U8   | Message Class      |
| 0x0003         | U8   | Message ID         |
| 0x0004         |      |                    |

## Data (Class 0x04)

### IMU Data (0x00)
| Payload Offset | Type | Description                          |
|----------------|------|--------------------------------------|
| 0x0000         | U8   | Packet Version = 1                   |
| 0x0001         | U8   | Reserved                             |
| 0x0002         | U64  | Data Timestamp (ms since Unix Epoch) |
| 0x000A         | F32  | Acc, X Axis (m/s/s)                  |
| 0x000E         | F32  | Acc, Y Axis (m/s/s)                  |
| 0x0012         | F32  | Acc, Z Axis (m/s/s)                  |
| 0x0016         | F32  | Gyro, X Axis (rad/s)                 |
| 0x001A         | F32  | Gyro, Y Axis (rad/s)                 |
| 0x001E         | F32  | Gyro, Z Axis (rad/s)                 |
| 0x0022         | F32  | Mag, X Axis (mT)                     |
| 0x0026         | F32  | Mag, Y Axis (mT)                     |
| 0x002A         | F32  | Mag, Z Axis (mT)                     |
| 0x002E         |      |                                      |

### Uncompressed 8-bit Audio Data (0x01)
| Payload Offset | Type  | Description                          |
|----------------|-------|--------------------------------------|
| 0x0000         | U8    | Packet Version = 1                   |
| 0x0001         | U8    | Number of Channels [C]               |
| 0x0002         | U16   | Number of samples per channel [N]    |
| 0x0004         | U64   | Data Timestamp (ms since Unix Epoch) |
| 0x000C         | U8[N] | Channel 1 Samples                    |
| 0x000C + N     | U8[N] | Channel 2 Samples                    |
| ...            |       |                                      |
| 0x000C + CN    |       |                                      |

### Uncompressed 16-bit Audio Data (0x02)
| Payload Offset | Type   | Description                          |
|----------------|--------|--------------------------------------|
| 0x0000         | U8     | Packet Version = 1                   |
| 0x0001         | U8     | Number of Channels [C]               |
| 0x0002         | U16    | Number of samples per channel [N]    |
| 0x0004         | U64    | Data Timestamp (ms since Unix Epoch) |
| 0x000C         | U16[N] | Channel 1 Samples                    |
| 0x000C + 2N    | U16[N] | Channel 2 Samples                    |
| ...            |        |                                      |
| 0x000C + 2CN   |        |                                      |

### Raw Data Stream (0xF0)
This is to facilitate custom and complex data types not covered by existing basic data types.  The Data ID is an 8-bit value unique to the endpoint that specifies what the data format is.  Other devices can discover this by polling the Raw Data Stream Configuration and retrieving the data ID MIME types.

| Payload Offset | Type | Description                          |
|----------------|------|--------------------------------------|
| 0x0000         | U8   | Packet Version = 1                   |
| 0x0001         | U8   | Data ID                              |
| 0x0002         | U64  | Data Timestamp (ms since Unix Epoch) |
| 0x000A         | U16  | Data Length (N)                      |
| 0x000C         | B[N] | Data                                 |
| 0x000C + N     |      |                                      |

### Raw File Header (ID 0xFC)
This is sent by the device (source) with the file to the device (target) to put the file onto.  When the target receives this packet, it should acknowledge the packet with a Raw File CTS packet within 5 seconds.  If the source does not receive a Raw File CTS packet within 5 seconds, it may reissue the Raw File Header packet no earlier than 8 seconds after it sent the first Raw File Header packet.

The source device shall assign a session unique file ID that will facilitate multiple file transfers simultaneously.  The target device must be able to handle multiple sources with identical file IDs.  File IDs may be reused once the original file transfer is complete.

The filename specified by the source shall be the base filename, including extension.  The target device shall prepend the datetime code to this, and place it in the appropriate directory.  The source device shall not specify in the filename a path or datetime code.

The file length shall be the number of Raw File Data packets expected.  Each Raw File Data packet can support at most 65483 bytes of data, and can also transmit fewer than 65483 bytes of data.

| Payload Offset | Type | Description                          |
|----------------|------|--------------------------------------|
| 0x0000         | U8   | Packet Version = 1                   |
| 0x0001         | U8   | File ID                              |
| 0x0002         | U16  | Filename Length (N)                  |
| 0x0004         | U16  | MIME Type Length (M)                 |
| 0x0006         | U64  | File Length (packets)                |
| 0x000E         | U64  | File Timestamp (ms since Unix Epoch) |
| 0x0016         | C[N] | Filename                             |
| 0x0016 + N     | C[M] | MIME Type                            |
| 0x0016 + N + M |      |                                      |

### Raw File CTS (ID 0xFD)
This is sent by the target device to the source device to acknowledge the Raw File Header packet.  Once the target has sent the Raw File CTS packet with an ACK, it shall expect a Raw File Data Packet within 30 seconds.  After 30 seconds, if the target does not receive a Raw File Data Packet, the target shall issue a Raw File CTS packet with a NACK, and consider the file ID available for further file transfers.

| Payload Offset | Type | Description        |
|----------------|------|--------------------|
| 0x0000         | U8   | Packet Version = 1 |
| 0x0001         | U8   | File ID            |
| 0x0002         | U8   | ACK                |
| 0x0003         |      |                    |

### Raw File ACK/NACK (ID 0xFE)
This is sent by the target device to the source device to acknowledge the receipt of a Raw File Data packet.  The target device shall expect the Raw File Data packets in sequence.  If the target device receives a Raw File Data packet out of sequence in a way that indicates it has missed a packet, it shall issue both a Raw File ACK for the packet that was received properly, and a Raw File NACK for the packet that was not received.  If the target device is expecting the next packet, but does not receive any Raw File Data packets for 30 seconds, then the target device shall issue a Raw File NACK for the next packet.  If the target device is only expecting retransmitted packets, and does not receive any Raw File Data packets for 30 seconds, then the target device shall issue a Raw File CTS NACK packet and discard the received file.

| Payload Offset | Type | Description        |
|----------------|------|--------------------|
| 0x0000         | U8   | Packet Version = 1 |
| 0x0001         | U8   | File ID            |
| 0x0002         | U16  | Packet Sequence    |
| 0x0004         | U8   | ACK                |
| 0x0005         |      |                    |

### Raw File (ID 0xFF)
This is sent by the source device to the target device to transfer data.  The source device shall send the Raw File Data packets sequentially without waiting for the corresponding Raw File ACK/NACK.  The source device shall also resend a Raw File Data packet in response to a Raw File NACK packet.

| Payload Offset | Type | Description           |
|----------------|------|-----------------------|
| 0x0000         | U8   | Packet Version = 1    |
| 0x0001         | U8   | File ID               |
| 0x0002         | U16  | Packet Sequence       |
| 0x0004         | U64  | Packet Length (bytes) |
| 0x000C         | B[N] | Data                  |
| 0x000C + N     |      |                       |
